In [3]:
import numpy as np
from tensorflow.keras.layers import Layer
from tensorflow.keras.initializers import RandomNormal
from tensorflow.keras.backend import squeeze, tanh, dot, expand_dims, sum, softmax

# Define a more distinctive input
input_data = np.array([[[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24],[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24],
                        ]])

# Define the Attention Layer
class AttentionLayer(Layer):
    def __init__(self, **kwargs):
        super(AttentionLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        self.W = self.add_weight(name="att_weight", shape=(input_shape[-1], 1), initializer=RandomNormal(mean=0.0, stddev=1.0))
        self.b = self.add_weight(name="att_bias", shape=(input_shape[1], 1), initializer="zeros")
        super(AttentionLayer, self).build(input_shape)

    def call(self, x):
        print("Shape of X is:--->", x.shape)
        print("Shape of W is: ------->", self.W.shape)
        et = squeeze(tanh(dot(x, self.W) + self.b), axis=-1)
        print("et value is", et.numpy())
        print("Step 1: Calculate tanh(dot(x, W) + b)")
        print("Result shape:", et.shape)
        print("Weights (W):")
        print(self.W.numpy())
        print("Bias (b):")
        print(self.b.numpy())
        
        at = softmax(et)
        print("\nStep 2: Calculate softmax")
        print("Result shape:", at.shape)
        print("Attention weights:")
        print(at.numpy())
        
        at = expand_dims(at, axis=-1)
        print("\nStep 3: Expand dimensions of attention weights")
        print("Result shape:", at.shape)
        
        output = x * at
        print("\nStep 4: Multiply input by attention weights")
        print("Result shape:", output.shape)
        
        output = sum(output, axis=1)
        print("\nStep 5: Sum along axis 1")
        print("Final output shape:", output.shape)
        
        return output

    def compute_output_shape(self, input_shape):
        return (input_shape[0], input_shape[-1])

# Create an instance of the AttentionLayer
attention_layer = AttentionLayer()

# Apply the AttentionLayer to the input data
output_data = attention_layer(input_data)

print("\nInput data:")
print(input_data)
print("\nOutput data:")
print(output_data.numpy())


Shape of X is:---> (1, 2, 24)
Shape of W is: -------> (24, 1)
et value is [[-1. -1.]]
Step 1: Calculate tanh(dot(x, W) + b)
Result shape: (1, 2)
Weights (W):
[[ 0.98674273]
 [-0.17824568]
 [-2.0293376 ]
 [ 0.19803815]
 [ 0.79331344]
 [ 0.25545767]
 [-1.774217  ]
 [ 1.3488773 ]
 [-0.09669409]
 [-1.4060137 ]
 [ 3.1828892 ]
 [ 0.5232729 ]
 [-1.8090416 ]
 [ 0.31025654]
 [-1.567651  ]
 [-2.2930984 ]
 [ 2.0591543 ]
 [-0.25074744]
 [-1.0286402 ]
 [-0.28456938]
 [ 0.49673414]
 [-0.05148069]
 [ 1.0968968 ]
 [ 0.2238658 ]]
Bias (b):
[[0.]
 [0.]]

Step 2: Calculate softmax
Result shape: (1, 2)
Attention weights:
[[0.5 0.5]]

Step 3: Expand dimensions of attention weights
Result shape: (1, 2, 1)

Step 4: Multiply input by attention weights
Result shape: (1, 2, 24)

Step 5: Sum along axis 1
Final output shape: (1, 24)

Input data:
[[[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
   24]
  [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
   24]]]

Output da

In [4]:
import numpy as np
import pandas as pd
from keras.models import Model
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, Flatten, Dropout, LSTM, Bidirectional, concatenate, Reshape, Layer
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report, confusion_matrix

# Custom Attention Layer
class AttentionLayer(Layer):
    def __init__(self, **kwargs):
        super(AttentionLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        self.W = self.add_weight(name="att_weight", shape=(input_shape[-1], 1), initializer="normal")
        self.b = self.add_weight(name="att_bias", shape=(input_shape[1], 1), initializer="zeros")
        super(AttentionLayer, self).build(input_shape)

    def call(self, x):
        et = K.squeeze(K.tanh(K.dot(x, self.W) + self.b), axis=-1)
        at = K.softmax(et)
        at = K.expand_dims(at, axis=-1)
        output = x * at
        return K.sum(output, axis=1)

    def compute_output_shape(self, input_shape):
        return (input_shape[0], input_shape[-1])

def readData(filename, readX, readY):
    file = open(filename, "r")
    for line in file:
        line.rstrip()
        temp = line.split(",")
        readX.append(float(temp[0]))
        dataSet = [float(t.rstrip()) for t in temp[1:]]
        readY.append(dataSet)
    file.close()

def initiate(a, c, d):
    s = len(a)
    t = len(a[1])
    i = 1
    var = []
    x = s - 1
    shape = (c, d)
    for x in range(0, x + 1):
        iarray = np.empty([1, t])
        iarray = np.array(a[x])
        imatrix = np.reshape(iarray, shape)
        var.append(imatrix)
    varar = np.asarray(var)
    return varar

def initiatec(list):
    narray = np.asarray(list)
    return narray

trainingX = []
trainingY = []
testX = []
testY = []

readData("Datasets/Lightning2_NOISE/Lightning2_TRAIN_NOISE50.csv", trainingX, trainingY)
readData("Datasets/Lightning2_NOISE/Lightning2_TEST_NOISE50.csv", testX, testY)

X_train = initiate(trainingY, 1, 637)
Y_train = initiatec(trainingX)

X_test = initiate(testY, 1, 637)
Y_test = initiatec(testX)

print('Training data shape : ', X_train.shape, Y_train.shape)
print('Testing data shape : ', X_test.shape, Y_test.shape)

classes = np.unique(Y_train)
nClasses = len(classes)

print('Total number of outputs : ', nClasses)
print('Output classes : ', classes)

train_X = X_train.reshape(-1, 1, 637, 1)
test_X = X_test.reshape(-1, 1, 637, 1)

print(train_X.shape, test_X.shape)

train_X = train_X.astype('float32')
test_X = test_X.astype('float32')

Y_train_one_hot = pd.get_dummies(Y_train)
Y_test_one_hot = pd.get_dummies(Y_test)

train_X, valid_X, train_label, valid_label = train_test_split(train_X, Y_train_one_hot, test_size=0.2, random_state=13)

print(train_X.shape)
print(valid_X.shape)
print(train_label.shape)
print(valid_label.shape)

batchsize = 2
epochs = 220
num_classes = 2

# Define the input shape for Conv2D layers
conv_input_shape = (1, 637, 1)

# Define the input layer for Conv2D layers
conv_input_layer = Input(shape=conv_input_shape)

# Adding Conv2D and MaxPooling2D layers for spatial features
conv_layer = Conv2D(6, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu')(conv_input_layer)
conv_layer = MaxPooling2D((2, 2), padding='same', strides=(1,1))(conv_layer)
conv_layer = Dropout(0.5)(conv_layer)
conv_layer = Conv2D(12, kernel_size=(3,3), padding='same', strides=(1,1), activation='relu')(conv_layer)
conv_layer = MaxPooling2D(pool_size=(2, 2), padding='same', strides=(1,1))(conv_layer)
conv_layer = Dropout(0.6)(conv_layer)
conv_output = Flatten()(conv_layer)

# Define the input shape for BiLSTM layer
lstm_input_shape = (1, 637)

# Define the input layer for BiLSTM layer
lstm_input_layer = Input(shape=lstm_input_shape)

# Adding BiLSTM layer with attention
lstm_layer = Bidirectional(LSTM(12, return_sequences=True))(lstm_input_layer)

attention_output = AttentionLayer()(lstm_layer)

# Concatenate the output of Conv2D and attention layer
concatenated_layers = concatenate([conv_output, attention_output])

# Dense layers
dense_layer = Dense(12, activation='sigmoid')(concatenated_layers)
dense_layer = Dense(12, activation='sigmoid')(dense_layer)
output_layer = Dense(num_classes, activation='sigmoid')(dense_layer)

# Define the model with both input branches and output
model = Model(inputs=[conv_input_layer, lstm_input_layer], outputs=output_layer)

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Print the model summary
print(model.summary())

# Define early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=25, verbose=1)

print('Train...')
# Train the model with early stopping
train = model.fit([train_X, train_X], train_label, batch_size=batchsize, epochs=epochs, verbose=1, validation_data=([valid_X, valid_X], valid_label),callbacks=[early_stopping] )

# Evaluate the model
test_eval = model.evaluate([test_X, test_X], Y_test_one_hot, verbose=1)

print('Test loss:', test_eval[0])
print('Test accuracy:', test_eval[1])

# Predict the classes for test data
predicted_classes = model.predict([test_X, test_X])

# Convert predictions to one-hot encoded format
predicted_classes_one_hot = pd.get_dummies(np.argmax(predicted_classes, axis=1))

# Compute the confusion matrix
confusion_mtx = confusion_matrix(Y_test_one_hot.values.argmax(axis=1), predicted_classes_one_hot.values.argmax(axis=1))
print('Confusion Matrix:')
print(confusion_mtx)

# Compute precision, recall, and F1-score
print(classification_report(Y_test_one_hot, predicted_classes_one_hot))


Training data shape :  (59, 1, 637) (59,)
Testing data shape :  (60, 1, 637) (60,)
Total number of outputs :  2
Output classes :  [0. 1.]
(59, 1, 637, 1) (60, 1, 637, 1)
(47, 1, 637, 1)
(12, 1, 637, 1)
(47, 2)
(12, 2)


Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)    │ (None, 1, 637, 1)         │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_2 (Conv2D)             │ (None, 1, 637, 6)         │              60 │ input_layer_2[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ max_pooling2d_2               │ (None, 1, 637, 6)         │               0 │ conv2d_2[0][0]             │
│ (MaxPooling2D)                │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dropout_2 (Dropout)           │ (None, 1, 637, 6)         │               0 │ max_pooling2d_2[0][0]      │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_3 (Conv2D)             │ (None, 1, 637, 12)        │             660 │ dropout_2[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ max_pooling2d_3               │ (None, 1, 637, 12)        │               0 │ conv2d_3[0][0]             │
│ (MaxPooling2D)                │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ input_layer_3 (InputLayer)    │ (None, 1, 637)            │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dropout_3 (Dropout)           │ (None, 1, 637, 12)        │               0 │ max_pooling2d_3[0][0]      │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ bidirectional_1               │ (None, 1, 24)             │          62,400 │ input_layer_3[0][0]        │
│ (Bidirectional)               │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ flatten_1 (Flatten)           │ (None, 7644)              │               0 │ dropout_3[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ attention_layer_2             │ (None, 24)                │              25 │ bidirectional_1[0][0]      │
│ (AttentionLayer)              │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ concatenate_1 (Concatenate)   │ (None, 7668)              │               0 │ flatten_1[0][0],           │
│                               │                           │                 │ attention_layer_2[0][0]    │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_3 (Dense)               │ (None, 12)                │          92,028 │ concatenate_1[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_4 (Dense)               │ (None, 12)                │             156 │ dense_3[0][0]              │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_5 (Dense)               │ (None, 2)                 │              2

 Total params: 155,355 (606.86 KB)

 Trainable params: 155,355 (606.86 KB)

 Non-trainable params: 0 (0.00 B)

None
Train...
Epoch 1/220


NameError: Exception encountered when calling AttentionLayer.call().

[1mname 'K' is not defined[0m

Arguments received by AttentionLayer.call():
  • x=tf.Tensor(shape=(None, 1, 24), dtype=float32)